# Expanding the measurement of culture with a sample of two billion humans

## Replication Data and Code

This notebook provides all the steps to replicate the results of our paper [Expanding the measurement of culture with a sample of two billion humans](https://doi.org/10.1098/rsif.2022.0085) published in the *Journal of the Royal Society Interface 19:20220085* (2022).

# Setup and Pre-requisites

Let's start by importing the required packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.font_manager as fm
from scipy.cluster import hierarchy as hch
from scipy.spatial import distance as dst
import matplotlib.patches as mpatches

# Package

The code of the function linkage() from package scipy.cluster.hierarchy is copied and edited to allow for using Ward with Cosine distance.

In [2]:
from scipy.cluster.hierarchy import linkage, ward, fcluster
import scipy.spatial.distance as distance
from scipy.spatial.distance import pdist, squareform

# Configuring d3

In [3]:
from IPython.display import HTML, Javascript, display

def configure_d3():
    display(Javascript("""
    require.config({
      paths: {
        d3: "https://d3js.org/d3.v4.min"
      }
    })"""))


configure_d3()

<IPython.core.display.Javascript object>

# Creating dendrograms

# To see the figure better start an html server and open the html

python -m http.server 8000

 ## All countries (225 countries)

In [4]:
def create_d3_dendogram_countries(df, fbdist, leaf_color=None, save_csv=True):
    """
        Returns schema for D3 Radial Dendrogram Plot
    """
    Z = ward(squareform(fbdist.values))

    rootnode, node_list = hch.to_tree(Z, rd=True)
    N = len(node_list) - 1

    geneology = dict()
    node_hierarchy = list()
    node_culture = list()
    node_continent = list()
    for node in node_list[::-1]:
        id_ = N - node.id
        if id_ == 0:
            geneology[id_] = str(id_)

        if id_ not in geneology:
            for node2 in node_list:
                if (node2.count != 1):
                    if (node.id == node2.left.id) or (node.id == node2.right.id):
                        id2 = N - node2.id
                        if node.dist == 0:
                            row = df.iloc[node.id]
                            country_name = row["Country"]
                            geneology[id_] = geneology[id2] + "." + country_name
                        else:
                            geneology[id_] = geneology[id2] + "." + str(id_)
                        break

        hierarchy = geneology[id_]
        if node.dist == 0:
            row = df.iloc[node.id]
            culture = row["categories"]
            continent = row["Continent"]
            node_hierarchy.append([hierarchy, culture, continent])
        else:
            node_hierarchy.append([hierarchy, None, None])

    d3_dendo_tree_df = pd.DataFrame(node_hierarchy)
    d3_dendo_tree_df.columns = ["id", "culture", "continent"]


    if save_csv:
        if leaf_color:
            d3_dendo_tree_df.to_csv('data/Dendrograms/dendrogram_Countries_all.csv')
        else:
            d3_dendo_tree_df.to_csv('data/Dendrograms/dendrogram.csv')

    return d3_dendo_tree_df

In [5]:
def addLabelColumns(df):
    cContinent = pd.read_csv('data/Dendrograms/countryInfo.csv', keep_default_na=False, na_values='')
    cCulture = pd.read_csv('data/Dendrograms/cultures.csv', keep_default_na=False, na_values='')
    cCountry = pd.read_csv('data/Dendrograms/ISO.csv',header=None, keep_default_na=False, na_values='')
    cCountry.columns = ["ISO","Country"]
    cClasses = cContinent[["ISO","ISO3","Continent"]].merge(cCulture,left_on="ISO3",right_on="countries",how='left')
    cClasses = cCountry[["ISO","Country"]].merge(cClasses,on="ISO",how='left')
    df2 = df.merge(cClasses[["ISO","Country","Continent","categories"]],left_on="ISO_CODE",right_on="ISO",how='left')
    df3 = df2[[df2.columns[0]]+list(df2.columns[-3:])+list(df2.columns[1:-4])].copy()
    df3['Country'] = df3['Country'].str.replace('.', '', regex=True)
    return df3

In [6]:
def filterCountries(df):
    filterList = list(df["Country"])
    df = df[df["Country"].isin(filterList)].copy()
    return df

In [7]:
fbdist = pd.read_stata('./data/OriginalData/FBCosDist.dta')
df = fbdist[['ISO_CODE_1']].drop_duplicates().reset_index(drop=True).copy()
df.columns = ['ISO_CODE']
fbdist = pd.pivot_table(fbdist, index='ISO_CODE_1', values='FBDist', columns='ISO_CODE_2')
fbdist.head()

ISO_CODE_2,AD,AE,AF,AG,AI,AL,AM,AO,AR,AS,...,VN,VU,WF,WS,XK,YE,YT,ZA,ZM,ZW
ISO_CODE_1,,,,,,,,,,,,,,,,,,,,,
AD,0.000000,0.233552,0.279233,0.227161,0.262021,0.216498,0.216255,0.229230,0.172900,0.274975,...,0.306917,0.226712,0.501356,0.231697,0.248052,0.272517,0.230188,0.205212,0.251032,0.264787
AE,0.233552,0.000000,0.184372,0.208990,0.298919,0.191062,0.171257,0.229923,0.200425,0.266877,...,0.256087,0.199313,0.499252,0.197320,0.220158,0.165144,0.214104,0.153619,0.184970,0.196901
AF,0.279233,0.184372,0.000000,0.256625,0.343870,0.224006,0.215051,0.238547,0.242429,0.297504,...,0.286739,0.217843,0.515573,0.223076,0.257178,0.175659,0.243067,0.203272,0.197842,0.205862
AG,0.227161,0.208990,0.256625,0.000000,0.215265,0.220900,0.198870,0.221629,0.202979,0.234386,...,0.293903,0.191093,0.493427,0.188906,0.254762,0.249615,0.214830,0.167668,0.190915,0.208627
AI,0.262021,0.298919,0.343870,0.215265,0.000000,0.300259,0.289365,0.299985,0.280734,0.275508,...,0.363204,0.263063,0.503569,0.268914,0.328159,0.335492,0.269510,0.265601,0.296988,0.313952


In [8]:
df = addLabelColumns(df)
df = filterCountries(df)

In [9]:
create_d3_dendogram_countries(df, fbdist, leaf_color="Culture")

,id,culture,continent
0,0,None,None
1,0.1,None,None
2,0.2,None,None
3,0.1.3,None,None
4,0.2.4,None,None
...,...,...,...
444,0.1.3.13.22.27.36.42.49.Anguilla,NaN,NA
445,0.1.3.13.22.31.67.94.109.Antigua and Barbuda,NaN,NA
446,0.2.4.15.20.29.39.70.Afghanistan,NaN,AS
447,0.2.4.15.20.29.41.192.206.United Arab Emirates,Islamic,AS


In [10]:
%%html

<style>
    body {
      font-family: 'Roboto Condensed', sans-serif;
    }
    .node circle  {
      fill: #999;
    }

    .node circle.cluster-one {
      fill: #CC6677;
    }

    .node circle.cluster-two {
      fill: #4477AA;
    }

    .node circle.cluster-three {
      fill: #DDCC77;
    }

    .node circle.cluster-four {
      fill: #2CA25F;
    }

    .node circle.cluster-five {
      fill: #DD1C77;
    }

    .node circle.cluster-six{
      fill: #D95F0E;
    }


    .node text {
      font-size: 10px;
      line-height: 1.4;
    }

    .light-brown {
      fill: #8c564b;
    }

    .grey {
      fill: #7f7f7f;
    }

    .magenta {
      fill: #e377c2;
    }

    .bright-green {
      fill: #2ca02c;
    }

    .purple {
      fill: #9467bd;
    }

    .light-military-green {
      fill: #bcbd22;
    }

    .light-red {
      fill: #d62728;
    }

    .orange {
      fill: #ff7f0e;
    }

    .cyan {
      fill: #17becf;
    }


    .node--internal text {
      text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
    }

    .link {
      fill: none;
      stroke: #555;
      stroke-opacity: 0.4;
      stroke-width: 2.5px;
    }

    .link.cluster-one {
      stroke: #CC6677;
    }

    .link.cluster-two {
      stroke: #4477AA;
    }

    .link.cluster-three {
      stroke: #DDCC77;
    }

    .link.cluster-four {
      stroke: #2CA25F;
    }

    .link.cluster-five {
      stroke: #DD1C77;
    }

    .link.cluster-six{
      stroke: #D95F0E;
    }

</style>
<body>

    <svg id="img_countries_all" width="1200" height="1200"></svg>

    <script>
      var width = 900,
          height = 900;

    require(['d3'], function (d3) {

      var svg = d3.select("#img_countries_all"),
          g = svg.append("g").attr("transform", "translate(" + ((width / 2)+80) +
                              "," + (height / 2 + 20) + ")"),
          legend = svg.append("g");


      var stratify = d3.stratify()
                       .parentId(function(d)
                          { return d.id.substring(0, d.id.lastIndexOf(".")); });

      var cluster = d3.cluster()
                      .size([360, width / 2 - 120]);

      function project(x, y) {
        var angle = (x - 90) / 180 * Math.PI, radius = y;
        return [radius * Math.cos(angle), radius * Math.sin(angle)];
      }


      d3.csv("data/Dendrograms/dendrogram_Countries_all.csv", function(error, data) {
        if (error) throw error;

        var root = stratify(data).sort(function(a, b) {
                        return a.height - b.height || a.id.localeCompare(b.id);
                      });

        cluster(root);

        var link = g.selectAll(".link")
                    .data(root.descendants().slice(1))
                    .enter().append("path")
                    .attr("d", function(d) {
                      return "M" + project(d.x, d.y)
                          + "C" + project(d.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, d.parent.y);
                    });

      link.attr("class", function(d) {
              var id = d.data.id;
              if (id[2]== "1") {
                  return "link cluster-one";
              } else if (id[2] == "2") {
                  return "link cluster-two";
              } else {
                return "link";
              }


            });

        var node = g.selectAll(".node")
                    .data(root.descendants())
                    .enter().append("g")
                    .attr("class", function(d) {
                      return "node" + (d.children ? " node--internal" : " node--leaf");
                    })
                    .attr("transform", function(d) {
                      return "translate(" + project(d.x, d.y) + ")";
                    });

        var leaf = g.selectAll(".node--leaf")
                    .append("circle")
                    .attr("r", 3)
                    .classed("cluster-one", function(d){
                      var id = d.data.id;
                      return (id[2] == "1");
                    })
                    .classed("cluster-two", function(d){
                      var id = d.data.id;
                      return (id[2]== "2") ;
                    });

        node.append("text")
            .attr("dy", "0.31em")
            .attr("x", function(d) {
              return d.x < 180 === !d.children ? 6 : -6;
            })
            .style("text-anchor", function(d) {
              return d.x < 180 === !d.children ? "start" : "end";
            })
            .attr("transform", function(d) {
              return "rotate(" + (d.x < 180 ? d.x - 90 : d.x + 90) + ")";
            })
            .text(function(d) {
              if (typeof d.children == "undefined") {
                  return d.id.substring(d.id.lastIndexOf(".") + 1);
              }
            })
            .attr("class", function(d){
              var id = d.data.id;
              if (id.slice(2,5) == "2.4")  console.log("slice",id.slice(2,10));

              if (id.slice(2,8)== "2.4.15") {
                  return "orange";
              } else if (id.slice(2,8) == "2.4.12") {
                  return "bright-green";
              } else if (id.slice(2,10) == "1.3.6.37") {
                   return "light-red";
              } else if (id.slice(2,12) == "1.3.6.9.17") {
                return "purple";
              } else if (id.slice(2,15) == "1.3.6.9.19.47") {
                return "light-brown";
              } else if (id.slice(2,18) == "1.3.6.9.19.21.66") {
                return "black";
              } else if (id.slice(2,20) == "1.3.6.9.19.21.32.6") {
                return "magenta";
              } else if (id.slice(2,24) == "1.3.6.9.19.21.32.48.64") {
                return "grey";
              } else if (id.slice(2,30) == "1.3.6.9.19.21.32.48.57.68.90") {
                return "light-military-green";
              } else if (id.slice(2,30) == "1.3.6.9.19.21.32.48.57.68.92") {
                return "cyan";
              } else {
                return "black";
              }


            });
      });
    });            
    </script>
</body>

In [11]:
%%writefile Dendrogram_All_Countries.html

<!DOCTYPE html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-beta/css/bootstrap.min.css" integrity="sha384-/Y6pD6FV/Vv2HJnA6t+vslU6fwYXjCFtcEpHbNJ0lyAFsXTsjBbfaDjzALeQsN6M" crossorigin="anonymous">

    <link href="https://fonts.googleapis.com/css?family=Roboto+Condensed" rel="stylesheet">

    <!-- <link rel="stylesheet" href="dendrogram.css" /> -->


<style>
    body {
      font-family: 'Roboto Condensed', sans-serif;
    }
    .node circle  {
      fill: #999;
    }

    .node circle.cluster-one {
      fill: #CC6677;
    }

    .node circle.cluster-two {
      fill: #4477AA;
    }

    .node circle.cluster-three {
      fill: #DDCC77;
    }

    .node circle.cluster-four {
      fill: #2CA25F;
    }

    .node circle.cluster-five {
      fill: #DD1C77;
    }

    .node circle.cluster-six{
      fill: #D95F0E;
    }


    .node text {
      font-size: 10px;
      line-height: 1.4;
    }

    .light-brown {
      fill: #8c564b;
    }

    .grey {
      fill: #7f7f7f;
    }

    .magenta {
      fill: #e377c2;
    }

    .bright-green {
      fill: #2ca02c;
    }

    .purple {
      fill: #9467bd;
    }

    .light-military-green {
      fill: #bcbd22;
    }

    .light-red {
      fill: #d62728;
    }

    .orange {
      fill: #ff7f0e;
    }

    .cyan {
      fill: #17becf;
    }


    .node--internal text {
      text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
    }

    .link {
      fill: none;
      stroke: #555;
      stroke-opacity: 0.4;
      stroke-width: 2.5px;
    }

    .link.cluster-one {
      stroke: #CC6677;
    }

    .link.cluster-two {
      stroke: #4477AA;
    }

    .link.cluster-three {
      stroke: #DDCC77;
    }

    .link.cluster-four {
      stroke: #2CA25F;
    }

    .link.cluster-five {
      stroke: #DD1C77;
    }

    .link.cluster-six{
      stroke: #D95F0E;
    }

    </style>
  </head>
  <body>

    <svg id="img_US_regions" width="1200" height="1200"></svg>
    <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.11.0/umd/popper.min.js" integrity="sha384-b/U6ypiBEHpOf/4+1nzFpr53nxSS+GLCkfwBdFNTxtclqqenISfwAzpKaMNFNmj4" crossorigin="anonymous"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-beta/js/bootstrap.min.js" integrity="sha384-h0AbiXch4ZDo7tp9hKZ4TsHbi047NrKGLO3SEJAg45jXxnGIfYzk4Si90RDIqNm1" crossorigin="anonymous"></script>
    <script src="https://d3js.org/d3.v4.min.js"></script>

    <svg id="img_countries_all" width="1200" height="1200"></svg>

    <script>
      var width = 900,
          height = 900;

    //require(['d3'], function (d3) {

      var svg = d3.select("#img_countries_all"),
          g = svg.append("g").attr("transform", "translate(" + ((width / 2)+80) +
                              "," + (height / 2 + 20) + ")"),
          legend = svg.append("g");


      var stratify = d3.stratify()
                       .parentId(function(d)
                          { return d.id.substring(0, d.id.lastIndexOf(".")); });

      var cluster = d3.cluster()
                      .size([360, width / 2 - 120]);

      function project(x, y) {
        var angle = (x - 90) / 180 * Math.PI, radius = y;
        return [radius * Math.cos(angle), radius * Math.sin(angle)];
      }


      d3.csv("data/Dendrograms/dendrogram_Countries_all.csv", function(error, data) {
        if (error) throw error;

        var root = stratify(data).sort(function(a, b) {
                        return a.height - b.height || a.id.localeCompare(b.id);
                      });

        cluster(root);

        var link = g.selectAll(".link")
                    .data(root.descendants().slice(1))
                    .enter().append("path")
                    .attr("d", function(d) {
                      return "M" + project(d.x, d.y)
                          + "C" + project(d.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, d.parent.y);
                    });

      link.attr("class", function(d) {
              var id = d.data.id;
              if (id[2]== "1") {
                  return "link cluster-one";
              } else if (id[2] == "2") {
                  return "link cluster-two";
              } else {
                return "link";
              }


            });

        var node = g.selectAll(".node")
                    .data(root.descendants())
                    .enter().append("g")
                    .attr("class", function(d) {
                      return "node" + (d.children ? " node--internal" : " node--leaf");
                    })
                    .attr("transform", function(d) {
                      return "translate(" + project(d.x, d.y) + ")";
                    });

        var leaf = g.selectAll(".node--leaf")
                    .append("circle")
                    .attr("r", 3)
                    .classed("cluster-one", function(d){
                      var id = d.data.id;
                      return (id[2] == "1");
                    })
                    .classed("cluster-two", function(d){
                      var id = d.data.id;
                      return (id[2]== "2") ;
                    });

        node.append("text")
            .attr("dy", "0.31em")
            .attr("x", function(d) {
              return d.x < 180 === !d.children ? 6 : -6;
            })
            .style("text-anchor", function(d) {
              return d.x < 180 === !d.children ? "start" : "end";
            })
            .attr("transform", function(d) {
              return "rotate(" + (d.x < 180 ? d.x - 90 : d.x + 90) + ")";
            })
            .text(function(d) {
              if (typeof d.children == "undefined") {
                  return d.id.substring(d.id.lastIndexOf(".") + 1);
              }
            })
            .attr("class", function(d){
              var id = d.data.id;
              if (id.slice(2,5) == "2.4")  console.log("slice",id.slice(2,10));

              if (id.slice(2,8)== "2.4.15") {
                  return "orange";
              } else if (id.slice(2,8) == "2.4.12") {
                  return "bright-green";
              } else if (id.slice(2,10) == "1.3.6.37") {
                   return "light-red";
              } else if (id.slice(2,12) == "1.3.6.9.17") {
                return "purple";
              } else if (id.slice(2,15) == "1.3.6.9.19.47") {
                return "light-brown";
              } else if (id.slice(2,18) == "1.3.6.9.19.21.66") {
                return "black";
              } else if (id.slice(2,20) == "1.3.6.9.19.21.32.6") {
                return "magenta";
              } else if (id.slice(2,24) == "1.3.6.9.19.21.32.48.64") {
                return "grey";
              } else if (id.slice(2,30) == "1.3.6.9.19.21.32.48.57.68.90") {
                return "light-military-green";
              } else if (id.slice(2,30) == "1.3.6.9.19.21.32.48.57.68.92") {
                return "cyan";
              } else {
                return "black";
              }


            });
      });
    //});            
    </script>
</body>

Writing Dendrogram_All_Countries.html


## Benchmark countries only (161 countries)

In [12]:
def create_d3_dendogram_countries(df, fbdist, leaf_color=None, save_csv=True):
    """
        Returns schema for D3 Radial Dendrogram Plot
    """
    Z = ward(squareform(fbdist.values))

    rootnode, node_list = hch.to_tree(Z, rd=True)
    N = len(node_list) - 1

    geneology = dict()
    node_hierarchy = list()
    node_culture = list()
    node_continent = list()
    for node in node_list[::-1]:
        id_ = N - node.id
        if id_ == 0:
            geneology[id_] = str(id_)

        if id_ not in geneology:
            for node2 in node_list:
                if (node2.count != 1):
                    if (node.id == node2.left.id) or (node.id == node2.right.id):
                        id2 = N - node2.id
                        if node.dist == 0:
                            row = df.iloc[node.id]
                            country_name = row["Country"]
                            geneology[id_] = geneology[id2] + "." + country_name
                        else:
                            geneology[id_] = geneology[id2] + "." + str(id_)
                        break

        hierarchy = geneology[id_]
        if node.dist == 0:
            row = df.iloc[node.id]
            culture = row["categories"]
            continent = row["Continent"]
            node_hierarchy.append([hierarchy, culture, continent])
        else:
            node_hierarchy.append([hierarchy, None, None])

    d3_dendo_tree_df = pd.DataFrame(node_hierarchy)
    d3_dendo_tree_df.columns = ["id", "culture", "continent"]


    if save_csv:
        if leaf_color:
            d3_dendo_tree_df.to_csv('data/Dendrograms/dendrogram_Countries_benchmark.csv')
        else:
            d3_dendo_tree_df.to_csv('data/Dendrograms/dendrogram.csv')

    return d3_dendo_tree_df

In [13]:
def filterCountries(df):
    filterList = list(pd.read_csv('data/Dendrograms/namesBenchmark.csv',header=None, keep_default_na=False, na_values='')[0])
    df = df[df["Country"].isin(filterList)].copy()
    return df

In [14]:
fbdist = pd.read_stata('../Expanding-Measurement-Culture-Facebook-JRSI/data/OriginalData/FBCosDist.dta')
df = fbdist[['ISO_CODE_1']].drop_duplicates().reset_index(drop=True).copy()
df.columns = ['ISO_CODE']
df = addLabelColumns(df)
df = filterCountries(df)
fbdist = fbdist.loc[(fbdist.ISO_CODE_1.isin(df.ISO_CODE))]
fbdist = fbdist.loc[(fbdist.ISO_CODE_2.isin(df.ISO_CODE))]
fbdist = pd.pivot_table(fbdist, index='ISO_CODE_1', values='FBDist', columns='ISO_CODE_2')
create_d3_dendogram_countries(df, fbdist, leaf_color="Culture")

,id,culture,continent
0,0,None,None
1,0.1,None,None
2,0.1.2,None,None
3,0.3,None,None
4,0.1.2.4,None,None
...,...,...,...
314,0.3.8.10.13.82.105.Angola,NaN,AF
315,0.3.6.18.30.42.91.Armenia,Orthodox,AS
316,0.1.2.5.7.21.130.Albania,Orthodox,EU
317,0.3.6.12.17.34.Afghanistan,NaN,AS


In [15]:
%%html

<style>
    body {
      font-family: 'Roboto Condensed', sans-serif;
    }
    .node circle  {
      fill: #999;
    }

    .node circle.cluster-one {
      fill: #CC6677;
    }

    .node circle.cluster-two {
      fill: #4477AA;
    }

    .node circle.cluster-three {
      fill: #DDCC77;
    }

    .node circle.cluster-four {
      fill: #2CA25F;
    }

    .node circle.cluster-five {
      fill: #DD1C77;
    }

    .node circle.cluster-six{
      fill: #D95F0E;
    }


    .node text {
      font-size: 15px;
      line-height: 1.4;
    }

    .light-brown {
      fill: #8c564b;
    }

    .grey {
      fill: #7f7f7f;
    }

    .magenta {
      fill: #e377c2;
    }

    .bright-green {
      fill: #2ca02c;
    }

    .purple {
      fill: #9467bd;
    }

    .light-military-green {
      fill: #bcbd22;
    }

    .light-red {
      fill: #d62728;
    }

    .orange {
      fill: #ff7f0e;
    }

    .cyan {
      fill: #17becf;
    }


    .node--internal text {
      text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
    }

    .link {
      fill: none;
      stroke: #555;
      stroke-opacity: 0.4;
      stroke-width: 2.5px;
    }

    .link.cluster-one {
      stroke: #CC6677;
    }

    .link.cluster-two {
      stroke: #4477AA;
    }

    .link.cluster-three {
      stroke: #DDCC77;
    }

    .link.cluster-four {
      stroke: #2CA25F;
    }

    .link.cluster-five {
      stroke: #DD1C77;
    }

    .link.cluster-six{
      stroke: #D95F0E;
    }

</style>
<body>

    <svg id="img_countries_benchmark" width="1200" height="1200"></svg>

    <script>
      var width = 900,
          height = 900;

    require(['d3'], function (d3) {

      var svg = d3.select("#img_countries_benchmark"),
          g = svg.append("g").attr("transform", "translate(" + ((width / 2)+80) +
                              "," + (height / 2 + 20) + ")"),
          legend = svg.append("g");


      var stratify = d3.stratify()
                       .parentId(function(d)
                          { return d.id.substring(0, d.id.lastIndexOf(".")); });

      var cluster = d3.cluster()
                      .size([360, width / 2 - 120]);

      function project(x, y) {
        var angle = (x - 90) / 180 * Math.PI, radius = y;
        return [radius * Math.cos(angle), radius * Math.sin(angle)];
      }


      d3.csv("data/Dendrograms/dendrogram_Countries_benchmark.csv", function(error, data) {
        if (error) throw error;

        var root = stratify(data).sort(function(a, b) {
                        return a.height - b.height || a.id.localeCompare(b.id);
                      });

        cluster(root);

        var link = g.selectAll(".link")
                    .data(root.descendants().slice(1))
                    .enter().append("path")
                    .attr("d", function(d) {
                      return "M" + project(d.x, d.y)
                          + "C" + project(d.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, d.parent.y);
                    });

      link.attr("class", function(d) {
              var id = d.data.id;
              if (id[2]== "1") {
                  return "link cluster-one";
              } else if (id[2] == "3") {
                  return "link cluster-two";
              } else {
                return "link";
              }


            });

        var node = g.selectAll(".node")
                    .data(root.descendants())
                    .enter().append("g")
                    .attr("class", function(d) {
                      return "node" + (d.children ? " node--internal" : " node--leaf");
                    })
                    .attr("transform", function(d) {
                      return "translate(" + project(d.x, d.y) + ")";
                    });

        var leaf = g.selectAll(".node--leaf")
                    .append("circle")
                    .attr("r", 3)
                    .classed("cluster-one", function(d){
                      var id = d.data.id;
                      return (id[2] == "1");
                    })
                    .classed("cluster-two", function(d){
                      var id = d.data.id;
                      return (id[2]== "3") ;
                    });

        node.append("text")
            .attr("dy", "0.31em")
            .attr("x", function(d) {
              return d.x < 180 === !d.children ? 6 : -6;
            })
            .style("text-anchor", function(d) {
              return d.x < 180 === !d.children ? "start" : "end";
            })
            .attr("transform", function(d) {
              return "rotate(" + (d.x < 180 ? d.x - 90 : d.x + 90) + ")";
            })
            .text(function(d) {
              if (typeof d.children == "undefined") {
                  return d.id.substring(d.id.lastIndexOf(".") + 1);
              }
            })
            .attr("class", function(d){
              var id = d.data.id;
              console.log("slice",id.slice(2,20));

              if (id.slice(2,5)== "3.6") {
                  return "orange";
              } else if (id.slice(2,5) == "3.8") {
                  return "bright-green";
              } else if (id.slice(2,6) == "1.13") {
                   return "light-red";
              } else if (id.slice(2,7) == "1.2.4") {
                return "purple";
              } else if (id.slice(2,10) == "1.2.5.21") {
                return "light-brown";
              } else if (id.slice(2,12) == "1.2.5.7.15") {
                return "black";
              } else if (id.slice(2,14) == "1.2.5.7.9.29") {
                return "magenta";
              } else if (id.slice(2,17) == "1.2.5.7.9.22.31") {
                return "grey";
              } else if (id.slice(2,20) == "1.2.5.7.9.22.33.51") {
                return "light-military-green";
              } else if (id.slice(2,20) == "1.2.5.7.9.22.33.52") {
                return "cyan";
              } else {
                return "black";
              }


            });
      });
    });
    </script>
</body>

In [16]:
%%writefile Dendrogram_All_Countries_Benchmark.html

<!DOCTYPE html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-beta/css/bootstrap.min.css" integrity="sha384-/Y6pD6FV/Vv2HJnA6t+vslU6fwYXjCFtcEpHbNJ0lyAFsXTsjBbfaDjzALeQsN6M" crossorigin="anonymous">

    <link href="https://fonts.googleapis.com/css?family=Roboto+Condensed" rel="stylesheet">

    <!-- <link rel="stylesheet" href="dendrogram.css" /> -->


<style>
    body {
      font-family: 'Roboto Condensed', sans-serif;
    }
    .node circle  {
      fill: #999;
    }

    .node circle.cluster-one {
      fill: #CC6677;
    }

    .node circle.cluster-two {
      fill: #4477AA;
    }

    .node circle.cluster-three {
      fill: #DDCC77;
    }

    .node circle.cluster-four {
      fill: #2CA25F;
    }

    .node circle.cluster-five {
      fill: #DD1C77;
    }

    .node circle.cluster-six{
      fill: #D95F0E;
    }


    .node text {
      font-size: 10px;
      line-height: 1.4;
    }

    .light-brown {
      fill: #8c564b;
    }

    .grey {
      fill: #7f7f7f;
    }

    .magenta {
      fill: #e377c2;
    }

    .bright-green {
      fill: #2ca02c;
    }

    .purple {
      fill: #9467bd;
    }

    .light-military-green {
      fill: #bcbd22;
    }

    .light-red {
      fill: #d62728;
    }

    .orange {
      fill: #ff7f0e;
    }

    .cyan {
      fill: #17becf;
    }


    .node--internal text {
      text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
    }

    .link {
      fill: none;
      stroke: #555;
      stroke-opacity: 0.4;
      stroke-width: 2.5px;
    }

    .link.cluster-one {
      stroke: #CC6677;
    }

    .link.cluster-two {
      stroke: #4477AA;
    }

    .link.cluster-three {
      stroke: #DDCC77;
    }

    .link.cluster-four {
      stroke: #2CA25F;
    }

    .link.cluster-five {
      stroke: #DD1C77;
    }

    .link.cluster-six{
      stroke: #D95F0E;
    }

    </style>
  </head>
  <body>

    <svg id="img_US_regions" width="1200" height="1200"></svg>
    <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.11.0/umd/popper.min.js" integrity="sha384-b/U6ypiBEHpOf/4+1nzFpr53nxSS+GLCkfwBdFNTxtclqqenISfwAzpKaMNFNmj4" crossorigin="anonymous"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-beta/js/bootstrap.min.js" integrity="sha384-h0AbiXch4ZDo7tp9hKZ4TsHbi047NrKGLO3SEJAg45jXxnGIfYzk4Si90RDIqNm1" crossorigin="anonymous"></script>
    <script src="https://d3js.org/d3.v4.min.js"></script>

    <svg id="img_countries_benchmark" width="1200" height="1200"></svg>

    <script>
      var width = 900,
          height = 900;

    //require(['d3'], function (d3) {

      var svg = d3.select("#img_countries_benchmark"),
          g = svg.append("g").attr("transform", "translate(" + ((width / 2)+80) +
                              "," + (height / 2 + 20) + ")"),
          legend = svg.append("g");


      var stratify = d3.stratify()
                       .parentId(function(d)
                          { return d.id.substring(0, d.id.lastIndexOf(".")); });

      var cluster = d3.cluster()
                      .size([360, width / 2 - 120]);

      function project(x, y) {
        var angle = (x - 90) / 180 * Math.PI, radius = y;
        return [radius * Math.cos(angle), radius * Math.sin(angle)];
      }


      d3.csv("data/Dendrograms/dendrogram_Countries_benchmark.csv", function(error, data) {
        if (error) throw error;

        var root = stratify(data).sort(function(a, b) {
                        return a.height - b.height || a.id.localeCompare(b.id);
                      });

        cluster(root);

        var link = g.selectAll(".link")
                    .data(root.descendants().slice(1))
                    .enter().append("path")
                    .attr("d", function(d) {
                      return "M" + project(d.x, d.y)
                          + "C" + project(d.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, d.parent.y);
                    });

      link.attr("class", function(d) {
              var id = d.data.id;
              if (id[2]== "1") {
                  return "link cluster-one";
              } else if (id[2] == "3") {
                  return "link cluster-two";
              } else {
                return "link";
              }


            });

        var node = g.selectAll(".node")
                    .data(root.descendants())
                    .enter().append("g")
                    .attr("class", function(d) {
                      return "node" + (d.children ? " node--internal" : " node--leaf");
                    })
                    .attr("transform", function(d) {
                      return "translate(" + project(d.x, d.y) + ")";
                    });

        var leaf = g.selectAll(".node--leaf")
                    .append("circle")
                    .attr("r", 3)
                    .classed("cluster-one", function(d){
                      var id = d.data.id;
                      return (id[2] == "1");
                    })
                    .classed("cluster-two", function(d){
                      var id = d.data.id;
                      return (id[2]== "3") ;
                    });

        node.append("text")
            .attr("dy", "0.31em")
            .attr("x", function(d) {
              return d.x < 180 === !d.children ? 6 : -6;
            })
            .style("text-anchor", function(d) {
              return d.x < 180 === !d.children ? "start" : "end";
            })
            .attr("transform", function(d) {
              return "rotate(" + (d.x < 180 ? d.x - 90 : d.x + 90) + ")";
            })
            .text(function(d) {
              if (typeof d.children == "undefined") {
                  return d.id.substring(d.id.lastIndexOf(".") + 1);
              }
            })
            .attr("class", function(d){
              var id = d.data.id;
              console.log("slice",id.slice(2,20));

              if (id.slice(2,5)== "3.6") {
                  return "orange";
              } else if (id.slice(2,5) == "3.8") {
                  return "bright-green";
              } else if (id.slice(2,6) == "1.13") {
                   return "light-red";
              } else if (id.slice(2,7) == "1.2.4") {
                return "purple";
              } else if (id.slice(2,10) == "1.2.5.21") {
                return "light-brown";
              } else if (id.slice(2,12) == "1.2.5.7.15") {
                return "black";
              } else if (id.slice(2,14) == "1.2.5.7.9.29") {
                return "magenta";
              } else if (id.slice(2,17) == "1.2.5.7.9.22.31") {
                return "grey";
              } else if (id.slice(2,20) == "1.2.5.7.9.22.33.51") {
                return "light-military-green";
              } else if (id.slice(2,20) == "1.2.5.7.9.22.33.52") {
                return "cyan";
              } else {
                return "black";
              }


            });
      });
    //});
    </script>
</body>

Writing Dendrogram_All_Countries_Benchmark.html


## Benchmark countries that are also in WVS data (74 countries)

In [17]:
def create_d3_dendogram_countries(df, fbdist, leaf_color=None, save_csv=True):
    """
        Returns schema for D3 Radial Dendrogram Plot
    """
    Z = ward(squareform(fbdist.values))

    rootnode, node_list = hch.to_tree(Z, rd=True)
    N = len(node_list) - 1

    geneology = dict()
    node_hierarchy = list()
    node_culture = list()
    node_continent = list()
    for node in node_list[::-1]:
        id_ = N - node.id
        if id_ == 0:
            geneology[id_] = str(id_)

        if id_ not in geneology:
            for node2 in node_list:
                if (node2.count != 1):
                    if (node.id == node2.left.id) or (node.id == node2.right.id):
                        id2 = N - node2.id
                        if node.dist == 0:
                            row = df.iloc[node.id]
                            country_name = row["Country"]
                            geneology[id_] = geneology[id2] + "." + country_name
                        else:
                            geneology[id_] = geneology[id2] + "." + str(id_)
                        break

        hierarchy = geneology[id_]
        if node.dist == 0:
            row = df.iloc[node.id]
            culture = row["categories"]
            continent = row["Continent"]
            node_hierarchy.append([hierarchy, culture, continent])
        else:
            node_hierarchy.append([hierarchy, None, None])

    d3_dendo_tree_df = pd.DataFrame(node_hierarchy)
    d3_dendo_tree_df.columns = ["id", "culture", "continent"]


    if save_csv:
        if leaf_color:
            d3_dendo_tree_df.to_csv('data/Dendrograms/dendrogram_Countries_WVS.csv')
        else:
            d3_dendo_tree_df.to_csv('data/Dendrograms/dendrogram.csv')

    return d3_dendo_tree_df

In [18]:
def filterCountries(df):
    filterList = list(pd.read_csv('data/Dendrograms/selectnames2.csv',header=None, keep_default_na=False, na_values='')[1])
    df = df[df["Country"].isin(filterList)].copy()
    return df

In [19]:
fbdist = pd.read_stata('../Expanding-Measurement-Culture-Facebook-JRSI/data/OriginalData/FBCosDist.dta')
df = fbdist[['ISO_CODE_1']].drop_duplicates().reset_index(drop=True).copy()
df.columns = ['ISO_CODE']
df = addLabelColumns(df)
df = filterCountries(df)
fbdist = fbdist.loc[(fbdist.ISO_CODE_1.isin(df.ISO_CODE))]
fbdist = fbdist.loc[(fbdist.ISO_CODE_2.isin(df.ISO_CODE))]
fbdist = pd.pivot_table(fbdist, index='ISO_CODE_1', values='FBDist', columns='ISO_CODE_2')
create_d3_dendogram_countries(df, fbdist, leaf_color="Culture")

,id,culture,continent
0,0,None,None
1,0.1,None,None
2,0.1.2,None,None
3,0.1.2.3,None,None
4,0.4,None,None
...,...,...,...
138,0.1.2.5.7.15.40.51.66.Australia,English,OC
139,0.1.2.5.7.16.27.34.63.67.Austria,Catholic,EU
140,0.1.9.13.23.33.61.Argentina,LatinAmerica,SA
141,0.4.11.19.42.Armenia,Orthodox,AS


In [20]:
%%html

<style>
    body {
      font-family: 'Roboto Condensed', sans-serif;
    }
    .node circle  {
      fill: #999;
    }

    .node circle.cluster-one {
      fill: #CC6677;
    }

    .node circle.cluster-two {
      fill: #4477AA;
    }

    .node circle.cluster-three {
      fill: #DDCC77;
    }

    .node circle.cluster-four {
      fill: #2CA25F;
    }

    .node circle.cluster-five {
      fill: #DD1C77;
    }

    .node circle.cluster-six{
      fill: #D95F0E;
    }


    .node text {
      font-size: 18px;
      line-height: 1.4;
    }

    .light-brown {
      fill: #8c564b;
    }

    .grey {
      fill: #7f7f7f;
    }

    .magenta {
      fill: #e377c2;
    }

    .bright-green {
      fill: #2ca02c;
    }

    .purple {
      fill: #9467bd;
    }

    .light-military-green {
      fill: #bcbd22;
    }

    .light-red {
      fill: #d62728;
    }

    .orange {
      fill: #ff7f0e;
    }

    .cyan {
      fill: #17becf;
    }


    .node--internal text {
      text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
    }

    .link {
      fill: none;
      stroke: #555;
      stroke-opacity: 0.4;
      stroke-width: 2.5px;
    }

    .link.cluster-one {
      stroke: #CC6677;
    }

    .link.cluster-two {
      stroke: #4477AA;
    }

    .link.cluster-three {
      stroke: #DDCC77;
    }

    .link.cluster-four {
      stroke: #2CA25F;
    }

    .link.cluster-five {
      stroke: #DD1C77;
    }

    .link.cluster-six{
      stroke: #D95F0E;
    }

</style>
<body>

    <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.11.0/umd/popper.min.js" integrity="sha384-b/U6ypiBEHpOf/4+1nzFpr53nxSS+GLCkfwBdFNTxtclqqenISfwAzpKaMNFNmj4" crossorigin="anonymous"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-beta/js/bootstrap.min.js" integrity="sha384-h0AbiXch4ZDo7tp9hKZ4TsHbi047NrKGLO3SEJAg45jXxnGIfYzk4Si90RDIqNm1" crossorigin="anonymous"></script>
    <script src="https://d3js.org/d3.v4.min.js"></script>
    <svg id="img_countries_WVS" width="1200" height="1200"></svg>

    <script>
      var width = 900,
          height = 900;
    require(['d3'], function (d3) {

      var svg = d3.select("#img_countries_WVS"),
          g = svg.append("g").attr("transform", "translate(" + ((width / 2)+80) +
                              "," + (height / 2 + 20) + ")"),
          legend = svg.append("g");


      var stratify = d3.stratify()
                       .parentId(function(d)
                          { return d.id.substring(0, d.id.lastIndexOf(".")); });

      var cluster = d3.cluster()
                      .size([360, width / 2 - 120]);

      function project(x, y) {
        var angle = (x - 90) / 180 * Math.PI, radius = y;
        return [radius * Math.cos(angle), radius * Math.sin(angle)];
      }


      d3.csv("data/Dendrograms/dendrogram_Countries_WVS.csv", function(error, data) {
        if (error) throw error;

        var root = stratify(data).sort(function(a, b) {
                        return a.height - b.height || a.id.localeCompare(b.id);
                      });

        cluster(root);

        var link = g.selectAll(".link")
                    .data(root.descendants().slice(1))
                    .enter().append("path")
                    .attr("d", function(d) {
                      return "M" + project(d.x, d.y)
                          + "C" + project(d.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, d.parent.y);
                    });

      link.attr("class", function(d) {
              var id = d.data.id;
              if (id[2]== "1") {
                  return "link cluster-one";
              } else if (id[2] == "4") {
                  return "link cluster-two";
              } else {
                return "link";
              }


            });

        var node = g.selectAll(".node")
                    .data(root.descendants())
                    .enter().append("g")
                    .attr("class", function(d) {
                      return "node" + (d.children ? " node--internal" : " node--leaf");
                    })
                    .attr("transform", function(d) {
                      return "translate(" + project(d.x, d.y) + ")";
                    });

        var leaf = g.selectAll(".node--leaf")
                    .append("circle")
                    .attr("r", 3)
                    .classed("cluster-one", function(d){
                      var id = d.data.id;
                      return (id[2] == "1");
                    })
                    .classed("cluster-two", function(d){
                      var id = d.data.id;
                      return (id[2]== "4") ;
                    });

        node.append("text")
            .attr("dy", "0.31em")
            .attr("x", function(d) {
              return d.x < 180 === !d.children ? 6 : -6;
            })
            .style("text-anchor", function(d) {
              return d.x < 180 === !d.children ? "start" : "end";
            })
            .attr("transform", function(d) {
              return "rotate(" + (d.x < 180 ? d.x - 90 : d.x + 90) + ")";
            })
            .text(function(d) {
              if (typeof d.children == "undefined") {
                  return d.id.substring(d.id.lastIndexOf(".") + 1);
              }
            })
            .attr("class", function(d){
              var id = d.data.id;
              console.log("slice",id.slice(2,15));

              if (id.slice(2,6)== "4.11") {
                  return "orange";
              } else if (id.slice(2,5) == "4.6") {
                  return "bright-green";
              } else if (id.slice(2,5) == "1.9") {
                   return "light-red";
              } else if (id.slice(2,7) == "1.2.3") {
                return "purple";
              } else if (id.slice(2,13) == "1.2.5.10.17") {
                return "light-brown";
              } else if (id.slice(2,13) == "1.2.5.10.14") {
                return "magenta";
              } else if (id.slice(2,12) == "1.2.5.7.15") {
                return "grey";
              } else if (id.slice(2,15) == "1.2.5.7.16.26") {
                return "light-military-green";
              } else if (id.slice(2,15) == "1.2.5.7.16.27") {
                return "cyan";
              } else {
                return "black";
              }

            });
      });
    });
    </script>
  </body>

In [21]:
%%writefile Dendrogram_Countries_WVS.html

<!DOCTYPE html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-beta/css/bootstrap.min.css" integrity="sha384-/Y6pD6FV/Vv2HJnA6t+vslU6fwYXjCFtcEpHbNJ0lyAFsXTsjBbfaDjzALeQsN6M" crossorigin="anonymous">

    <link href="https://fonts.googleapis.com/css?family=Roboto+Condensed" rel="stylesheet">

    <!-- <link rel="stylesheet" href="dendrogram.css" /> -->


<style>
    body {
      font-family: 'Roboto Condensed', sans-serif;
    }
    .node circle  {
      fill: #999;
    }

    .node circle.cluster-one {
      fill: #CC6677;
    }

    .node circle.cluster-two {
      fill: #4477AA;
    }

    .node circle.cluster-three {
      fill: #DDCC77;
    }

    .node circle.cluster-four {
      fill: #2CA25F;
    }

    .node circle.cluster-five {
      fill: #DD1C77;
    }

    .node circle.cluster-six{
      fill: #D95F0E;
    }


    .node text {
      font-size: 10px;
      line-height: 1.4;
    }

    .light-brown {
      fill: #8c564b;
    }

    .grey {
      fill: #7f7f7f;
    }

    .magenta {
      fill: #e377c2;
    }

    .bright-green {
      fill: #2ca02c;
    }

    .purple {
      fill: #9467bd;
    }

    .light-military-green {
      fill: #bcbd22;
    }

    .light-red {
      fill: #d62728;
    }

    .orange {
      fill: #ff7f0e;
    }

    .cyan {
      fill: #17becf;
    }


    .node--internal text {
      text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
    }

    .link {
      fill: none;
      stroke: #555;
      stroke-opacity: 0.4;
      stroke-width: 2.5px;
    }

    .link.cluster-one {
      stroke: #CC6677;
    }

    .link.cluster-two {
      stroke: #4477AA;
    }

    .link.cluster-three {
      stroke: #DDCC77;
    }

    .link.cluster-four {
      stroke: #2CA25F;
    }

    .link.cluster-five {
      stroke: #DD1C77;
    }

    .link.cluster-six{
      stroke: #D95F0E;
    }

    </style>
  </head>
  <body>

    <svg id="img_US_regions" width="1200" height="1200"></svg>
    <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.11.0/umd/popper.min.js" integrity="sha384-b/U6ypiBEHpOf/4+1nzFpr53nxSS+GLCkfwBdFNTxtclqqenISfwAzpKaMNFNmj4" crossorigin="anonymous"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-beta/js/bootstrap.min.js" integrity="sha384-h0AbiXch4ZDo7tp9hKZ4TsHbi047NrKGLO3SEJAg45jXxnGIfYzk4Si90RDIqNm1" crossorigin="anonymous"></script>
    <script src="https://d3js.org/d3.v4.min.js"></script>

    <svg id="img_countries_WVS" width="1200" height="1200"></svg>

    <script>
      var width = 900,
          height = 900;

    //require(['d3'], function (d3) {

      var svg = d3.select("#img_countries_WVS"),
          g = svg.append("g").attr("transform", "translate(" + ((width / 2)+80) +
                              "," + (height / 2 + 20) + ")"),
          legend = svg.append("g");


      var stratify = d3.stratify()
                       .parentId(function(d)
                          { return d.id.substring(0, d.id.lastIndexOf(".")); });

      var cluster = d3.cluster()
                      .size([360, width / 2 - 120]);

      function project(x, y) {
        var angle = (x - 90) / 180 * Math.PI, radius = y;
        return [radius * Math.cos(angle), radius * Math.sin(angle)];
      }


      d3.csv("data/Dendrograms/dendrogram_Countries_WVS.csv", function(error, data) {
        if (error) throw error;

        var root = stratify(data).sort(function(a, b) {
                        return a.height - b.height || a.id.localeCompare(b.id);
                      });

        cluster(root);

        var link = g.selectAll(".link")
                    .data(root.descendants().slice(1))
                    .enter().append("path")
                    .attr("d", function(d) {
                      return "M" + project(d.x, d.y)
                          + "C" + project(d.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, (d.y + d.parent.y) / 2)
                          + " " + project(d.parent.x, d.parent.y);
                    });

      link.attr("class", function(d) {
              var id = d.data.id;
              if (id[2]== "1") {
                  return "link cluster-one";
              } else if (id[2] == "4") {
                  return "link cluster-two";
              } else {
                return "link";
              }


            });

        var node = g.selectAll(".node")
                    .data(root.descendants())
                    .enter().append("g")
                    .attr("class", function(d) {
                      return "node" + (d.children ? " node--internal" : " node--leaf");
                    })
                    .attr("transform", function(d) {
                      return "translate(" + project(d.x, d.y) + ")";
                    });

        var leaf = g.selectAll(".node--leaf")
                    .append("circle")
                    .attr("r", 3)
                    .classed("cluster-one", function(d){
                      var id = d.data.id;
                      return (id[2] == "1");
                    })
                    .classed("cluster-two", function(d){
                      var id = d.data.id;
                      return (id[2]== "4") ;
                    });

        node.append("text")
            .attr("dy", "0.31em")
            .attr("x", function(d) {
              return d.x < 180 === !d.children ? 6 : -6;
            })
            .style("text-anchor", function(d) {
              return d.x < 180 === !d.children ? "start" : "end";
            })
            .attr("transform", function(d) {
              return "rotate(" + (d.x < 180 ? d.x - 90 : d.x + 90) + ")";
            })
            .text(function(d) {
              if (typeof d.children == "undefined") {
                  return d.id.substring(d.id.lastIndexOf(".") + 1);
              }
            })
            .attr("class", function(d){
              var id = d.data.id;
              console.log("slice",id.slice(2,15));

              if (id.slice(2,6)== "4.11") {
                  return "orange";
              } else if (id.slice(2,5) == "4.6") {
                  return "bright-green";
              } else if (id.slice(2,5) == "1.9") {
                   return "light-red";
              } else if (id.slice(2,7) == "1.2.3") {
                return "purple";
              } else if (id.slice(2,13) == "1.2.5.10.17") {
                return "light-brown";
              } else if (id.slice(2,13) == "1.2.5.10.14") {
                return "magenta";
              } else if (id.slice(2,12) == "1.2.5.7.15") {
                return "grey";
              } else if (id.slice(2,15) == "1.2.5.7.16.26") {
                return "light-military-green";
              } else if (id.slice(2,15) == "1.2.5.7.16.27") {
                return "cyan";
              } else {
                return "black";
              }

            });
      });
    //});
    </script>
</body>

Writing Dendrogram_Countries_WVS.html
